In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import tensorflow_datasets as tfds
from collections import Counter
import scipy.ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import gc
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tarfile
import urllib.request
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F
from scipy.stats import gaussian_kde
from numba import jit
import torch.nn as nn

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

limit = 1000

num_samples = 128  # Monte-Carlo Dropout 取樣次數
dropout_rate = 0.1

# bzq modifying
len_x_target = 20
len_y_target = 20
stride_x_target = 10
stride_y_target = 10

# mean, std proportion
alpha = 0.5

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 0
pixels = 0

In [ ]:
# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
class MCDropoutResNet(nn.Module):
    def __init__(self, dropout_p=0.5):
        super().__init__()
        # **載入預訓練的 ResNet50**
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

        # **在 ResNet 的 feature extractor 內加入 Dropout**
        self.dropout_layers = nn.ModuleList([
            nn.Dropout(p=dropout_p) for _ in range(4)  # 在 layer1, layer2, layer3, layer4 之後插入 Dropout
        ])

        # **在 FC 層前加入 Dropout，但保留原始權重**
        self.resnet.fc = nn.Sequential(
            nn.Dropout(p=dropout_p),
            nn.Linear(self.resnet.fc.in_features, 1000)
        )

        # **複製原始 FC 層的權重**
        pretrained_resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.resnet.fc[1].weight.data = pretrained_resnet.fc.weight.data.clone()
        self.resnet.fc[1].bias.data = pretrained_resnet.fc.bias.data.clone()

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        # **遍歷 ResNet 各層並加入 Dropout**
        x = self.resnet.layer1(x)
        x = self.dropout_layers[0](x)  # Dropout 在 layer1 之後

        x = self.resnet.layer2(x)
        x = self.dropout_layers[1](x)  # Dropout 在 layer2 之後

        x = self.resnet.layer3(x)
        x = self.dropout_layers[2](x)  # Dropout 在 layer3 之後

        x = self.resnet.layer4(x)
        x = self.dropout_layers[3](x)  # Dropout 在 layer4 之後

        # **進入 FC 層**
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.resnet.fc(x)

        return x
    
    def enable_dropout(self):
        """
        啟用模型內的 Dropout 層，使其在推論時仍然啟動
        """
        for m in self.modules():
            if isinstance(m, nn.Dropout):
                m.train()  # 讓 Dropout 在推論時仍作用
    
# 加載預訓練的ResNet50模型
#model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model = MCDropoutResNet(dropout_p=dropout_rate).to(device)
model.eval()

In [ ]:
# 得到了bzq 正確的函數，拿來做imagenet 正確的預測

def single_data_bzq_mask_preprocessing_imagenet(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x, :] *= magnification
    return new_data

def single_data_bzq_mask_preprocessing_imagenet_random_global(original_data, start_x, start_y, len_x, len_y, random_num_for_bzq_mask_imagenet):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data)
    random_num_for_bzq_mask_imagenet = random_num_for_bzq_mask_imagenet[:, :len_y, :len_x] 
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] = random_num_for_bzq_mask_imagenet
    return new_data

bzq = []
correct_predictions_imagenet = []
incorrect_predictions_imagenet = []
bzq_imagenet = []

#bzq = 0的時候，提取mask之下的softmax
correct_predictions_bzq_zero_softmax_mean = []
correct_predictions_bzq_zero_softmax_std = []
incorrect_predictions_bzq_zero_softmax_mean = []
incorrect_predictions_bzq_zero_softmax_std = []

# 生成隨機影像，形狀為 (limit, 3, 224, 224)
random_images = torch.rand(limit, 3, 224, 224)

# 生成存放 -1 的列表
negative_labels = [-1] * limit

# 建立 test_dataset 為 list of tuples
test_dataset = [(random_images[i], negative_labels[i]) for i in range(limit)]

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
'''model.eval()
predictions = []
output = model(test_dataset[1][0].float().unsqueeze(0).to(device))

softmax_output = F.softmax(output, dim=1).flatten()  # 轉換成機率
predictions.append(softmax_output.cpu().detach().numpy())
predictions = np.array(predictions)
mean_pred = predictions.mean(axis=0)  # 平均預測
mean_pred = mean_pred.flatten()
entropy = -np.sum(mean_pred * np.log(np.clip(mean_pred, 1e-10, 1)), axis=0)  # 防止 log(0) 錯誤
print(entropy)'''

In [ ]:
from scipy.stats import entropy
import torch
import torch.nn.functional as F
import numpy as np

predictions = []

if dropout_rate > 0.0:
    # 強制啟用 Dropout
    model.enable_dropout()  # 只啟用 Dropout 層

# **遍歷 test_loader 進行 Monte-Carlo Dropout**
for inputs, _ in test_loader:  # 使用 test_loader 來獲取輸入影像
    inputs = inputs.float().to(device)  # 移動到 GPU / CPU
    
    sample_predictions = []
    
    for _ in range(num_samples):
        output = model(inputs)  # 針對相同輸入取樣
        softmax_output = F.softmax(output, dim=1)  # 轉換成機率
        sample_predictions.append(softmax_output.cpu().detach().numpy())

    predictions.append(np.stack(sample_predictions, axis=0))  # (num_samples, 1, num_classes)
# **Monte-Carlo Dropout：計算均值與標準差**
predictions = np.array(predictions).squeeze(axis=2)  # 形成 (test_size, num_samples, num_classes)

mean_pred = predictions.mean(axis=1)  # 針對每個樣本取平均
std_pred = predictions.std(axis=1)  # 計算標準差（不確定性指標）

# **修正 entropy 計算**
entropy_values = [entropy(mean_pred[i]) for i in range(mean_pred.shape[0])]

print(entropy_values)

In [ ]:
print(test_dataset[0][0].size())
# 計算熵的分佈（分桶）
bins = np.linspace(min(entropy_values), max(entropy_values), 15) 
hist, bin_edges = np.histogram(entropy_values, bins=bins)  # 計算直方圖數據
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # 計算每個 bin 的中心點

# 繪製折線圖
plt.figure(figsize=(10, 5))
plt.plot(bin_centers, hist, linestyle='-', marker='o')

# 設定標題與軸標籤
plt.title("Entropy Distribution")
plt.xlabel("Entropy")
plt.ylabel("# of Examples")
plt.legend()
plt.grid(True)
plt.show()